<a href="https://colab.research.google.com/github/simecek/PseudoDNA_Generator/blob/master/colabs/experiments/Intergenomic_MarkovModel_KMER3_v0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Setup

In [1]:
!pip install git+https://github.com/ML-Bioinfo-CEITEC/rbp

  Cloning https://github.com/ML-Bioinfo-CEITEC/rbp to /tmp/pip-req-build-6z0ox72e
  Running command git clone -q https://github.com/ML-Bioinfo-CEITEC/rbp /tmp/pip-req-build-6z0ox72e
  Created wheel for rbp: filename=rbp-0.1.3-py3-none-any.whl size=11790 sha256=932af79b27e1dda0dc480723e89555ef3a8b046afb1923c4f85e17fdbac8a4bb
  Stored in directory: /tmp/pip-ephem-wheel-cache-bzaiqsel/wheels/90/37/7c/214b3142d6e6b059ea0d2600c41a5453ab8ab0aef1b0d48e25
Successfully built rbp


In [14]:
import pandas as pd
import numpy as np
from rbp.random.markov_model import MarkovModel
from tqdm import tqdm, notebook
notebook.tqdm().pandas()

In [4]:
# Mount to your Google Drive allowing lesson files will be saved to your Drive location
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


## Data

In [6]:
dt = pd.read_csv("/content/drive/My Drive/data/random/intergenomic_seqs_50k.csv")
dt = dt[~dt.seq.str.contains("N")]   # just for sure

train = dt[dt.chr!="1"]
test = dt[dt.chr=="1"]

print(dt.shape, train.shape, test.shape)
dt.head()

(50000, 4) (46286, 4) (3714, 4)


,chr,start,end,seq
0,17,14239397,14239596,AACTGGGATTCACAGGAGCTTAATGGAGCACATGATGTTAAGTGAA...
1,X,90928374,90928573,GCTAGTTGTATGGTTAGCAGCAAGATATTTTTTCTCTCTGATCTTT...
2,3,104278717,104278916,GACTTTGTAGACTTGTGTGACCTGTGTGCCTCCCTCTCCCCCCAAA...
3,4,187089054,187089253,ATGTTAACACCAAATCAGTCCATCCTAATTATCACTCAAAAATCAA...
4,2,137742849,137743048,GCAGGAGCTCTATCTGTTTGGACTAGTTCAGCCCCATCTCTTTTGG...


## Markov Model

In [7]:
mm = MarkovModel(k=3)
mm.fit(train.seq)

In [9]:
def predict3(prev3):
    tmp = mm.kmer_pairs[mm.kmer_pairs.index.get_level_values('prev') == prev3].reset_index().drop(columns='prev').sort_values(0, ascending=False)
    return tmp.curr.iloc[0]

predict3("ACG")

'GGA'

In [11]:
next3 = {x: predict3(x) for x in mm.kmer_pairs.index.get_level_values('prev')}
next3['ACG']

'GGA'

In [12]:
def compare3(x, y):
    return int(x[0] == y[0]), int(x[1] == y[1]), int(x[2] == y[2])
compare3('GGG', 'AGG')

(0, 1, 1)

In [13]:
def stat3(s):
    sdf = pd.DataFrame({'prev': [s[i:i+3] for i in range(50,200-3,3)], 'next': [s[i:i+3] for i in range(53,200,3)]})
    sdf['nextpred'] = sdf.prev.apply(predict3)
    sdf[['c1', 'c2', 'c3']] = pd.DataFrame.from_records(sdf.apply(lambda row: compare3(row['next'], row['nextpred']), axis=1), columns=['c1', 'c2', 'c3'])
    return sdf[['c1', 'c2', 'c3']].mean(axis=0)

In [15]:
stats = test.seq.progress_apply(stat3)

In [16]:
stats.mean(axis=0)

c1    0.331778
c2    0.320283
c3    0.307535
dtype: float64